In [1]:
import geopandas as gpd, folium, requests, pandas as pd, fiona, branca, json, numpy as np

In [2]:
income = pd.read_csv("https://raw.githubusercontent.com/pri-data/50-states/master/data/income-counties-states-national.csv")
income.columns

Index(['fips', 'county', 'state', 'income-2015', 'income-1989a',
       'income-1989b', 'change'],
      dtype='object')

In [3]:
gdf = gpd.read_file("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json")
statesabbr=pd.read_json("https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_titlecase.json")
income = pd.read_csv("https://raw.githubusercontent.com/pri-data/50-states/master/data/income-counties-states-national.csv")
income['income-2015']=pd.to_numeric(income['income-2015'], errors='coerce')
income=income.groupby(by=['state'], as_index=False).median()
income['State']=pd.merge(income, statesabbr, left_on='state', right_on='abbreviation')['name']
income.head()
# income.dropna(subset=['income-2015'])[['state','income-2015']].groupby(by='state')

,state,fips,income-2015,income-1989b,change,State
0,AK,2154.0,62561.5,66619.55885,14.758367,Alaska
1,AL,1066.0,38721.5,38644.47667,2.779114,Alabama
2,AR,5074.0,37890.0,34699.07158,10.897394,Arkansas
3,AZ,4012.5,43810.0,39230.65642,NaN,Arizona
4,CA,6057.0,53341.0,50230.52704,6.716596,California


In [4]:

gdf1=gdf.merge(income, left_on='name', right_on='State', how='left').fillna(0)

In [5]:
gdf1.crs = fiona.crs.from_epsg(4326)
gdf1.head()

,id,name,density,geometry,state,fips,income-2015,income-1989b,change,State
0,01,Alabama,94.650,"POLYGON ((-87.359296 35.00118, -85.606675 34.9...",AL,1066.0,38721.5,38644.47667,2.779114,Alabama
1,02,Alaska,1.264,"(POLYGON ((-131.602021 55.117982, -131.569159 ...",AK,2154.0,62561.5,66619.55885,14.758367,Alaska
2,04,Arizona,57.050,"POLYGON ((-109.042503 37.000263, -109.04798 31...",AZ,4012.5,43810.0,39230.65642,0.000000,Arizona
3,05,Arkansas,56.430,"POLYGON ((-94.473842 36.501861, -90.152536 36....",AR,5074.0,37890.0,34699.07158,10.897394,Arkansas
4,06,California,241.700,"POLYGON ((-123.233256 42.006186, -122.378853 4...",CA,6057.0,53341.0,50230.52704,6.716596,California


In [6]:
gdf1=gdf1[gdf1['name']!='District of Columbia']

In [7]:
roads = gpd.read_file(r"/Users/jasonbaker/Downloads/tl_2016_us_primaryroads/tl_2016_us_primaryroads.shp")
roads = roads.to_crs(gdf1.crs)
interstates = roads.loc[roads['RTTYP']=='I']
interstates.loc[:,'geometry']=interstates.geometry.simplify(tolerance=0.01, preserve_topology=False)
interstates = interstates.dissolve(by='FULLNAME', aggfunc='first', as_index=False)
interstates.loc[:,'FULLNAME'] = interstates['FULLNAME'].str.replace('- ','-')
interstates.head()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,FULLNAME,geometry,LINEARID,RTTYP,MTFCC
0,E I-10,(LINESTRING (-97.84499699999999 29.61460000000...,1105261674551,I,S1100
1,E I-20,"LINESTRING (-98.970765 32.37466599999999, -98....",110206001615,I,S1100
2,E I-30,(LINESTRING (-96.58229599999999 32.84527599999...,1104487239695,I,S1100
3,E I-96,"(LINESTRING (-84.83710099999999 42.827011, -84...",110467146892,I,S1100
4,I--375,"LINESTRING (-82.65380499999999 27.777177, -82....",1105083070005,I,S1100


In [8]:
variable = 'income-2015'
gdf1=gdf1.sort_values(by=variable, ascending=True)
colormap = folium.LinearColormap(colors=["red","orange","yellow","green"],vmin=gdf1.loc[gdf1[variable]>0, variable].min(), 
                                 vmax=gdf1.loc[gdf1[variable]>0, variable].max()).to_step(n=5)
colormap.caption = "2015 US Median Household Income"

In [9]:
centroid=gdf1.geometry.centroid

m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=4, tiles='stamentoner')

folium.GeoJson(gdf1[['geometry','name',variable]],
               name="United States of America",
               style_function=lambda x: {"weight":2, 'color':'black','fillColor':colormap(x['properties'][variable]), 'fillOpacity':0.2},
              highlight_function=lambda x: {'weight':3, 'color':'black'},
               smooth_factor=2.0,
              tooltip=folium.features.Tooltip(fields=['name',variable,],
                                              aliases=['State','2015 Median Income'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
              ).add_to(m)


folium.GeoJson(interstates,
               name='US Interstate Highways',
               style_function=lambda x: {'weight':2,'color':'#5BDAE9','dashArray':'1 3'},
               highlight_function=lambda x: {'weight':5,'color':'yellow', 'dashArray':'1 7'},
               tooltip=folium.features.Tooltip(fields=['FULLNAME',],labels=False, toLocaleString=True),
               smooth_factor=5.0,
              ).add_to(m)


colormap.add_to(m)

folium.LayerControl(autoZIndex=False, collapsed=False).add_to(m)

m

